# Bereiten wir das Script vor

In [2]:
#Imports
import os #Files in listen abgreifen
import tqdm #anzeigen, wo wir stehen
import textract
import pandas as pd


## Vorbereitung

In [3]:
#Als Vorbereitung, machen aus dem Inhalt des Folders eine Liste. Es handelt sich hierbei
#um eine Liste aller pdfs im Ordner pdfs. Es sind die Swissmedic Recalls (#ImplantFiles)
pdfs = os.listdir('pdfs')

## Fortschritt anzeigen

In [8]:
from tqdm import tqdm
for i in tqdm(range(1000)): #100000000000
    elem = i

100%|██████████| 10000000/10000000 [00:03<00:00, 2849323.36it/s]


## Ein File auslesen

In [12]:
text = textract.process("pdfs/"+pdfs[1]) #method='pdfminer', encoding='ISO-8859-1'

## Alle Files auslesen

In [18]:
lst = []
mistakes = []
for pdf in tqdm(pdfs):
    try:
        text = textract.process("pdfs/"+pdf, method='pdfminer', encoding='ISO-8859-1')
        lst.append(text)
    except:
        mistakes.append(pdf)


 33%|███▎      | 27/82 [00:16<00:19,  2.83it/s]

Vk_20180517_12documents0.pdf



 48%|████▊     | 39/82 [00:25<00:40,  1.05it/s]

Vk_20180523_05documents2.pdf



 55%|█████▍    | 45/82 [00:28<00:16,  2.27it/s]

...


 72%|███████▏  | 59/82 [00:35<00:07,  3.06it/s]

Vk_20180530_11documents3.pdf
Vk_20180524_02documents0.pdf



 73%|███████▎  | 60/82 [00:35<00:06,  3.54it/s]

Vk_20180326_03documents0.pdf



100%|██████████| 82/82 [00:50<00:00,  2.73it/s]

## Bilder auslesen

In [21]:
text = textract.process("pdfs/"+pdfs[0], method='tesseract') #Nicht ideal, aber gut

## Alles zusammen

In [ ]:
#Rohe Analyse
for pdf in tqdm(pdfs):
    try:
        text = textract.process("pdfs/"+pdf, method='pdfminer', encoding='ISO-8859-1')
        text = text.decode("ISO-8859-1").replace("\n", " ")
        if len(text) < 40:
            text = textract.process("pdfs/"+pdf, method='tesseract', language='deu')
            text = text.decode("ISO-8859-1").replace("\n", " ")
            
        mini_dict = {'Text':text,
                     'File':pdf}
        lst.append(mini_dict)
    except:
        mini_dict = {'Text':'Fehlermeldung',
                     'File':pdf}
        lst.append(mini_dict)


 59%|█████▊    | 48/82 [00:37<00:40,  1.18s/it]

In [ ]:
#Erste Säuberung
df = pd.DataFrame(lst)
def date(elem):
    elem = elem[3:11]
    return elem
df['date'] = df['File'].apply(date)
df['date'] = pd.to_datetime(df['date'], format='%Y%m%d')
df.index = df['date']
#Suchen wir noch nach Implantaten
df['implant'] = df['Text'].str.contains('implant')
df.to_csv('datafile.csv')